# Toy GWAS in Google Colab (simulated but realistic)

This notebook simulates genotype data with mild population structure, generates a phenotype, and runs a GWAS twice:

1) **Unadjusted** (shows inflation)
2) **Adjusted for PCs** (reduces confounding)

It then makes **Manhattan** and **QQ** plots.

> Designed to run in a few minutes on Colab.

In [ ]:
#@title Setup
!pip -q install numpy pandas scipy statsmodels scikit-learn matplotlib

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

np.random.seed(42)

## 1) Simulate genotypes with mild population structure

We simulate two subpopulations with slightly different allele frequencies (a simple way to create confounding).

In [ ]:
#@title Simulate genotypes
n = 2000          # individuals
m = 12000         # SNPs (keep moderate for Colab speed)

# random minor allele frequencies
maf = np.random.uniform(0.05, 0.5, size=m)

# two populations (0/1)
pop = np.random.binomial(1, 0.5, size=n)

# allele-frequency shift between populations (small)
delta = np.random.normal(0, 0.03, size=m)  # mild structure
maf_pop1 = np.clip(maf + delta, 0.01, 0.99)
maf_pop0 = np.clip(maf - delta, 0.01, 0.99)

# genotype matrix G: n x m, coded 0/1/2
G = np.empty((n, m), dtype=np.int8)
for i in range(n):
    p = maf_pop1 if pop[i] == 1 else maf_pop0
    G[i, :] = np.random.binomial(2, p).astype(np.int8)

# quick QC: remove monomorphic SNPs (should be rare here)
var = G.var(axis=0)
keep = var > 0
G = G[:, keep]
maf = maf[keep]
m = G.shape[1]

print(f"Genotypes: n={n}, m={m}")

## 2) Create a phenotype

We pick a few causal SNPs, add a small population effect (confounding), and add noise.

In [ ]:
#@title Simulate phenotype
n_causal = 20
causal_idx = np.random.choice(m, size=n_causal, replace=False)
beta_causal = np.random.normal(0, 0.25, size=n_causal)

genetic_signal = G[:, causal_idx] @ beta_causal
pop_effect = 0.6 * (pop - pop.mean())          # confounding
noise = np.random.normal(0, 1.0, size=n)

y = (genetic_signal + pop_effect + noise)
y = (y - y.mean()) / y.std()

pheno = pd.DataFrame({"iid": np.arange(n), "pop": pop, "y": y})
pheno.head()

## 3) Compute PCs

In real GWAS, PCs are computed from genotype data to capture ancestry-related structure.

In [ ]:
#@title PCA
# standardize variants for PCA
G_std = (G - G.mean(axis=0)) / G.std(axis=0)
pca = PCA(n_components=5, random_state=42)
pcs = pca.fit_transform(G_std)

for k in range(5):
    pheno[f"PC{k+1}"] = pcs[:, k]

pheno[["pop","PC1","PC2","PC3"]].corr()

## 4) GWAS (fast linear regression)

We implement association testing by residualizing **y** and each SNP on covariates, then using the correlation test. This is fast and transparent for teaching.

- **Unadjusted:** no covariates
- **Adjusted:** covariates = PC1–PC5

In [ ]:
#@title Helper: residualize and run GWAS
def residualize(v, X):
    # v: (n,), X: (n,k) with intercept included
    beta, *_ = np.linalg.lstsq(X, v, rcond=None)
    return v - X @ beta

def gwas_linear(G, y, cov=None):
    n, m = G.shape
    if cov is None:
        X = np.ones((n,1))
    else:
        X = np.column_stack([np.ones(n), cov])
    y_r = residualize(y, X)

    # residualize each SNP on covariates (loop over chunks for memory)
    betas = np.zeros(m, dtype=float)
    ses = np.zeros(m, dtype=float)
    pvals = np.zeros(m, dtype=float)

    chunk = 2000
    for start in range(0, m, chunk):
        end = min(m, start + chunk)
        Gc = G[:, start:end].astype(float)

        # residualize SNPs
        # Solve (X'X)^{-1}X'G for each SNP using lstsq
        B, *_ = np.linalg.lstsq(X, Gc, rcond=None)  # (k+1) x chunk
        Gc_r = Gc - X @ B

        # association using simple regression y_r ~ Gc_r
        # beta = cov / var
        cov_y = (Gc_r * y_r[:, None]).sum(axis=0)
        var_g = (Gc_r ** 2).sum(axis=0)
        beta = cov_y / var_g

        # SE from residual variance
        resid = y_r[:, None] - Gc_r * beta[None, :]
        sigma2 = (resid**2).sum(axis=0) / (n - X.shape[1] - 1)
        se = np.sqrt(sigma2 / var_g)

        z = beta / se
        p = 2 * stats.norm.sf(np.abs(z))

        betas[start:end] = beta
        ses[start:end] = se
        pvals[start:end] = p

    return betas, ses, pvals

# run both
cov_pcs = pheno[[f"PC{k+1}" for k in range(5)]].to_numpy()
beta0, se0, p0 = gwas_linear(G, y, cov=None)
beta1, se1, p1 = gwas_linear(G, y, cov=cov_pcs)

print("Done.")

## 5) Summaries: inflation (λGC) and top hits

In [ ]:
#@title Summaries
def lambda_gc(pvals):
    chi2 = stats.chi2.isf(pvals, df=1)
    return np.median(chi2) / stats.chi2.ppf(0.5, df=1)

lam0 = lambda_gc(p0)
lam1 = lambda_gc(p1)

print(f"Lambda GC (unadjusted): {lam0:.3f}")
print(f"Lambda GC (PC-adjusted): {lam1:.3f}")

# top SNPs
top = np.argsort(p1)[:20]
top_df = pd.DataFrame({
    "snp_index": top,
    "beta_adj": beta1[top],
    "se_adj": se1[top],
    "p_adj": p1[top],
    "is_causal": np.isin(top, causal_idx)
}).sort_values("p_adj")

top_df

## 6) Manhattan + QQ plots

We generate fake chromosome/position coordinates (so the plots look like a real GWAS).

In [ ]:
#@title Create synthetic genomic coordinates
# assign SNPs to chromosomes
chroms = np.repeat(np.arange(1, 23), np.ceil(m / 22).astype(int))[:m]
# positions within chromosome
pos = np.zeros(m, dtype=int)
for c in range(1, 23):
    idx = np.where(chroms == c)[0]
    pos[idx] = np.sort(np.random.randint(1, 50_000_000, size=len(idx)))

# cumulative position for Manhattan
chrom_max = pd.Series(pos).groupby(chroms).max()
offsets = chrom_max.cumsum().shift(fill_value=0).to_dict()
x = pos + np.array([offsets[c] for c in chroms])

coord = pd.DataFrame({"chr": chroms, "pos": pos, "x": x, "p_unadj": p0, "p_adj": p1})
coord.head()

In [ ]:
#@title Manhattan plot (PC-adjusted)
df = coord.sort_values(["chr","pos"])
df["mlog10p"] = -np.log10(df["p_adj"].clip(1e-300, 1.0))

plt.figure(figsize=(12,4))
plt.scatter(df["x"], df["mlog10p"], s=4, alpha=0.7)
plt.axhline(-np.log10(5e-8), linestyle="--")  # genome-wide threshold
plt.xlabel("Genomic position (synthetic)")
plt.ylabel("-log10(P)")
plt.title("Toy GWAS Manhattan plot (PC-adjusted)")
plt.show()

In [ ]:
#@title QQ plots (unadjusted vs adjusted)
def qqplot(pvals, title):
    p = np.sort(pvals)
    exp = -np.log10((np.arange(1, len(p)+1) - 0.5) / len(p))
    obs = -np.log10(p.clip(1e-300, 1.0))
    plt.figure(figsize=(4.5,4.5))
    plt.scatter(exp, obs, s=6, alpha=0.7)
    mx = max(exp.max(), obs.max())
    plt.plot([0,mx],[0,mx], linestyle="--")
    plt.xlabel("Expected -log10(P)")
    plt.ylabel("Observed -log10(P)")
    plt.title(title)
    plt.show()

qqplot(p0, f"QQ plot (unadjusted), lambda={lam0:.2f}")
qqplot(p1, f"QQ plot (PC-adjusted), lambda={lam1:.2f}")

## Teaching notes (quick talking points)

- Why population structure inflates test statistics
- What PCs capture
- Manhattan vs QQ
- λGC and why it is not a perfect diagnostic (but useful for demos)
